In [37]:
!pip install vaderSentiment
import datasets
import requests
import numpy as np

In [65]:
dataset = datasets.load_dataset('imdb')

train_dataset = dataset['train']
test_dataset = dataset['test']

url = "https://raw.githubusercontent.com/cjhutto/vaderSentiment/master/vaderSentiment/vader_lexicon.txt"
response = requests.get(url)
text = response.text
lexicon_words = text.split()

Found cached dataset imdb (C:/Users/$0NT000-3G88GUGFR0B8/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 198.46it/s]


In [55]:
def lowercase_punctuation(text):
    newtext = text.lower()
    # remove <br /> tags
    newtext = newtext.replace('<br />', '')
    # remove punctuation
    newtext = ''.join([c if c not in "\"#$%&()*+,./:;<=>?@[\]^_`{|}~" else ' ' for c in newtext])
    return newtext.split()

In [56]:
def count_pronouns(word_list):
    count = 0
    for word in word_list:
        if word in ["i", "you", "we"]:
            count += 1
    return count

In [73]:
def lexicon_word_counter(word_list):
    positive_count = 0
    negative_count = 0
    lower_lexicon_value = -1
    upper_lexicon_value = 1
    for word in word_list:
        if word in lexicon_words:
            sentimental_value = 0
            if isinstance(lexicon_words[lexicon_words.index(word) + 1], str):
                continue
            if float(sentimental_value) < lower_lexicon_value:
                negative_count += 1
            elif float(sentimental_value) > upper_lexicon_value:
                positive_count += 1
    return positive_count, negative_count

In [75]:
def vector_generator(dataset):
    resulting_vector = []
    for obj in dataset:
        current_vector = []
        word_list = lowercase_punctuation(obj['text'])
        current_vector.append(1) if word_list.__contains__('no') else current_vector.append(0)
        current_vector.append(count_pronouns(word_list))
        current_vector.append(1) if word_list.__contains__('!') else current_vector.append(0)
        current_vector.append(np.log(len(word_list)))
        positive_count, negative_count = lexicon_word_counter(word_list)
        current_vector.append(positive_count)
        current_vector.append(negative_count)
        resulting_vector.append(current_vector)
    return resulting_vector

In [ ]:
print(vector_generator(train_dataset))